In [1]:
import os
import pandas as pd

domains = ["cooking", "diy"]

k = 10

for domain in domains:

    run_files_path = f"measurements/{domain}/run_files"
    run_files = os.listdir(run_files_path)

    annotation_set = set()

    for file in run_files:
        with open(os.path.join(os.getcwd(), run_files_path, file), "r") as f:
            # runs = [run for run in f]
            for run in f:
                run_components = " ".join(run.split(' ')[0:3])
                # print(run)
                if int(run.split(' ')[3]) <= k:
                    annotation_set.add(run_components)

    empty_annotations = sorted(annotation_set)

    queries = pd.read_csv(f'queries/{domain}.csv')

    print("annotations per topic", len(empty_annotations) / 100)

    annotations_list = []

    for annotations in empty_annotations:
        query_id, _, taskgraph_id = annotations.split(' ')
        # print(query_id)
        raw_query = queries[queries["id"] == query_id].iloc[0]["raw query"]
        annotations_list.append({
            "query-id": query_id,
            "raw query": raw_query,
            "taskgraph-id": taskgraph_id,
        })

    empty_annotations_path = f"measurements/{domain}/empty_annotations"
    df = pd.DataFrame(annotations_list)
    df.to_csv(os.path.join(os.getcwd(), empty_annotations_path, f"updated-empty_annotations-{k}.csv"), index=False)
# queries["id"]

# with open(os.path.join(os.getcwd(), empty_annotations_path, 'combined_annotations.csv'), "r") as f:
        


annotations per topic 60.21
annotations per topic 33.46


In [9]:
# get missing annotations
import csv 
updated_annotations_path = "measurements/diy/empty_annotations/updated-empty_annotations-10.csv"
new_annotations_path = "measurements/diy/empty_annotations/empty_annotations-10.csv"

with open(updated_annotations_path, "r") as f:
    updated_annotations = {line for line in f}
    
with open(new_annotations_path, "r") as f:
    annotations = {line for line in f}
diff_annotations =  list(updated_annotations.difference(annotations))
diff_annotations = [a.strip().split(',') for a in diff_annotations]
with open('measurements/diy/empty_annotations/difference-empty_annotations-10.csv','w') as result_file:
    wr = csv.writer(result_file, delimiter=',')
    wr.writerows(diff_annotations)
# print(diff_annotations)

# len(diff_annotations)


In [32]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [36]:
from models_indexes.marqo_model import MarqoModel
from models_indexes.abstract_model import AbstractModel

models = {}

# for domain in  ["DIY", "COOKING"]:
for domain in  ["DIY"]:
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance": AnceModel(domain = domain),
        # "colbert": ColbertModel(domain = domain),
        "marqo": MarqoModel(domain = domain),
    }
    
models["DIY"]["marqo"].search("fix a fridge")
# models["DIY"]["marqo"].search("fridge")
# models["DIY"]["marqo"].get_stats()
models["DIY"]["marqo"].get_single_document("diy+wikihow-offline+4e3c05e275eed0c1b2572ac770139c52")
# print("Creating run files ")
# for domain, index_models in models.items():
#     qs = queries[domain]
#     print(f"DOMAIN {domain}")
#     for model_name, model in index_models.items():
#         model.convert_search_results_to_run(qs)

{'hits': [], 'processingTimeMs': 35, 'query': 'fix a fridge', 'limit': 10}